In [1]:
# Set constants
DATASET = "Task 2 (10% Noise)"
ALLOW_ONLY_HP_MATCHES = True
DIFFERENCE_THRESHS = [1.05, 1.1, 1.5, 3, 4.5]

In [2]:
# Load data
import pandas as pd

data = pd.read_csv("latent_trials.csv")
data.head()

,Classifier,Based on AE,sigma,precision,recall,f-score,max_iter,auc,n_estimators,max_depth,min_samples_leaf,numEstimators,C,gamma,kernel,probability,algorithm,leaf_size,n_neighbors
0,ReconstructionThreshold,20230223203432,1.05,0.857143,0.3750,0.521739,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ReconstructionThreshold,20230223203432,1.10,0.833333,0.3125,0.454545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ReconstructionThreshold,20230223203432,1.50,1.000000,0.1250,0.222222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ReconstructionThreshold,20230223203432,3.00,0.000000,0.0000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ReconstructionThreshold,20230223203432,4.50,0.000000,0.0000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Add processed classifer column
data["Processed Classifier"] = [classifier + "_" + str(sigma) if classifier == "ReconstructionThreshold" else classifier for classifier, sigma in data[["Classifier", "sigma"]].values]
data = data.drop(columns=["Classifier"])

data["Dataset"] = [DATASET for _ in data.index]

In [4]:
# Add AE data

ae_data = pd.read_csv("ae_trials.csv", keep_default_na=False)
ae_data = ae_data[["name", "type", "input_size", "attempt", "leader"]]
ae_data["Primary Model"] = ["%d" % float(leader) if leader else "%d" % float(base) for base, leader in ae_data[["name", "leader"]].values]
ae_data = ae_data.drop(columns=["leader"])
ae_data.head()

,name,type,input_size,attempt,Primary Model
0,20230223203432,Adversarial AE,4096,0,20230223203432
1,20230223203614,Vanilla AE,4096,0,20230223203432
2,20230223203727,Adversarial AE,4096,1,20230223203432
3,20230223203909,Vanilla AE,4096,1,20230223203432
4,20230223204022,Adversarial AE,4096,2,20230223203432


In [5]:
# Convert AE data to dict and add to ll data

ae_data = {name: [ae_type, input_size, attempt, primary] for name, ae_type, input_size, attempt, primary in ae_data.values}
data = data[[name in ae_data.keys() for name in data["Based on AE"].values]]
data[["Type", "Input Size", "Attempt", "Primary AE Model"]] = [ae_data[name] for name in data["Based on AE"].values]

data[data["Processed Classifier"] == "Discriminator_Thresh_30"]["Type"]

9       Adversarial AE
41      Adversarial AE
73      Adversarial AE
105     Adversarial AE
137     Adversarial AE
             ...      
5321    Adversarial AE
5353    Adversarial AE
5385    Adversarial AE
5417    Adversarial AE
5449    Adversarial AE
Name: Type, Length: 169, dtype: object

In [6]:
# Create groups and check if hyper-params are the same
hyper_params_to_check = [
    "max_iter",
    "n_estimators",
    "max_depth",
    "min_samples_leaf",
    "numEstimators",
    "C",
    "gamma",
    "kernel",
    "probability",
    "algorithm",
    "leaf_size",
    "n_neighbors"
]

data_uniqueness = data[["Primary AE Model", "Processed Classifier"] + hyper_params_to_check]
data_uniqueness = data_uniqueness.groupby(["Primary AE Model", "Processed Classifier"], group_keys=False).nunique()
data_uniqueness["HP Match"] = [all([n<=1 for n in v]) for v in data_uniqueness.values]

data_uniqueness = {(i[0], i[1]): same_hp for i, same_hp in zip(data_uniqueness.index.values, data_uniqueness["HP Match"].values)}

data["HP Match"] = [data_uniqueness[b_ae, pc] for b_ae, pc in data[["Primary AE Model", "Processed Classifier"]].values]
data["Classifier"] = data["Processed Classifier"]
data = data.drop(columns=["Processed Classifier"])

data = data[["Dataset", "Type", "Input Size", "Primary AE Model", "Classifier", "Attempt", "HP Match", "precision", "recall", "f-score", "auc"]]

data.head()

,Dataset,Type,Input Size,Primary AE Model,Classifier,Attempt,HP Match,precision,recall,f-score,auc
0,Task 2 (10% Noise),Adversarial AE,4096,20230223203432,ReconstructionThreshold_1.05,0,True,0.857143,0.3750,0.521739,NaN
1,Task 2 (10% Noise),Adversarial AE,4096,20230223203432,ReconstructionThreshold_1.1,0,True,0.833333,0.3125,0.454545,NaN
2,Task 2 (10% Noise),Adversarial AE,4096,20230223203432,ReconstructionThreshold_1.5,0,True,1.000000,0.1250,0.222222,NaN
3,Task 2 (10% Noise),Adversarial AE,4096,20230223203432,ReconstructionThreshold_3.0,0,True,0.000000,0.0000,0.000000,NaN
4,Task 2 (10% Noise),Adversarial AE,4096,20230223203432,ReconstructionThreshold_4.5,0,True,0.000000,0.0000,0.000000,NaN


In [7]:
# Rename p, r and f1
data = data.rename(columns={"precision": "Precision", "recall": "Recall", "f-score": "F1-Score", "auc": "Area Under Curve"})

In [8]:
# Create final table
def row_max(df):
    max_f1 = df["F1-Score"].max()
    for r, f1 in zip(df.values, df["F1-Score"].values):
        if f1 == max_f1:
            return pd.DataFrame([r], columns=["Precision", "Recall", "F1-Score", "Area Under Curve", "Precision Standard Deviation", "Recall Standard Deviation", "F1-Score Standard Deviation", "Area Under Curve Standard Deviation"])
    

if ALLOW_ONLY_HP_MATCHES:
    data = data[data["HP Match"]]

data = data[["Dataset", "Type", "Input Size", "Primary AE Model", "Classifier", "Precision", "Recall", "F1-Score", "Area Under Curve"]]
data_std = data.groupby(["Dataset", "Type", "Input Size", "Primary AE Model", "Classifier"]).std()
data = data.groupby(["Dataset", "Type", "Input Size", "Primary AE Model", "Classifier"]).mean()

data[["Precision Standard Deviation", "Recall Standard Deviation", "F1-Score Standard Deviation", "Area Under Curve Standard Deviation"]] = data_std[["Precision", "Recall", "F1-Score", "Area Under Curve"]]


data = data.groupby(["Dataset", "Input Size", "Type", "Classifier"]).apply(row_max)

formated_strings = ["%.4f (%.4f, %.4f)" % (f, p, r) for p, r, f in data[["Precision", "Recall", "F1-Score"]].values]
data["Formated Results"] = formated_strings

data = data.sort_values(["Type", "F1-Score"], ascending=False)

data.head()

Precision  \
Dataset            Input Size Type       Classifier                                
Task 2 (10% Noise) 4096       Vanilla AE ExtraTreesClassifier       0        1.0   
                                         GradientBoostingClassifier 0        1.0   
                                         KNeighborsClassifier       0        1.0   
                                         LogisticRegression         0        1.0   
                                         RandomForestClassifier     0        1.0   

                                                                       Recall  \
Dataset            Input Size Type       Classifier                             
Task 2 (10% Noise) 4096       Vanilla AE ExtraTreesClassifier       0     1.0   
                                         GradientBoostingClassifier 0     1.0   
                                         KNeighborsClassifier       0     1.0   
                                         LogisticRegression         0     1.0   
                                         RandomForestClassifier     0     1.0   

                                                                       F1-Score  \
Dataset            Input Size Type       Classifier                               
Task 2 (10% Noise) 4096       Vanilla AE ExtraTreesClassifier       0       1.0   
                                         GradientBoostingClassifier 0       1.0   
                                         KNeighborsClassifier       0       1.0   
                                         LogisticRegression         0       1.0   
                                         RandomForestClassifier     0       1.0   

                                                                       Area Under Curve  \
Dataset            Input Size Type       Classifier                                       
Task 2 (10% Noise) 4096       Vanilla AE ExtraTreesClassifier       0               NaN   
                                         GradientBoostingClassifier 0               NaN   
                                         KNeighborsClassifier       0               NaN   
                                         LogisticRegression         0               NaN   
                                         RandomForestClassifier     0               NaN   

                                                                       Precision Standard Deviation  \
Dataset            Input Size Type       Classifier                                                   
Task 2 (10% Noise) 4096       Vanilla AE ExtraTreesClassifier       0                           0.0   
                                         GradientBoostingClassifier 0                           0.0   
                                         KNeighborsClassifier       0                           0.0   
                                         LogisticRegression         0                           0.0   
                                         RandomForestClassifier     0                           0.0   

                                                                       Recall Standard Deviation  \
Dataset            Input Size Type       Classifier                                                
Task 2 (10% Noise) 4096       Vanilla AE ExtraTreesClassifier       0                        0.0   
                                         GradientBoostingClassifier 0                        0.0   
                                         KNeighborsClassifier       0                        0.0   
                                         LogisticRegression         0                        0.0   
                                         RandomForestClassifier     0                        0.0   

                                                                       F1-Score Standard Deviation  \
Dataset            Input Size Type       Classifier                                                  
Task 2 (10% Noise) 4096       Vanilla AE ExtraTreesClassifier      

In [9]:
# Save final table

data.to_csv("Final LL Classifiers Table.csv")